In [2]:
import pandas as pd

import re
import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import date_fixer

In [3]:
df = pd.read_csv('daily8_2.csv')
new = date_fixer.make_datetime(df)

TRN_ARB_0297955 successful
TRN_ARB_0297957 successful
TRN_ARB_0297958 successful
TRN_ARB_0297960 successful
TRN_ARB_0297984 successful
TRN_ARB_0297985 successful
TRN_ARB_0298009 successful
TRN_ARB_0298010 successful
TRN_ARB_0298011 successful
TRN_ARB_0298015 successful
TRN_ARB_0298017 successful
TRN_ARB_0298018 successful
TRN_ARB_0298039 successful
TRN_ARB_0298057 successful
TRN_ARB_0298060 successful
TRN_ARB_0298061 successful
TRN_ARB_0298062 successful
TRN_ARB_0298063 successful
TRN_ARB_0298065 successful
TRN_ARB_0298067 successful
TRN_ARB_0298068 successful
TRN_ARB_0298071 successful
TRN_ARB_0298072 successful
TRN_ARB_0298073 successful
TRN_ARB_0298074 successful
TRN_ARB_0298088 successful
TRN_ARB_0298128 successful
TRN_ARB_0298130 successful
TRN_ARB_0298134 successful
TRN_ARB_0298153 successful
TRN_ARB_0298162 successful
TRN_ARB_0298163 successful
TRN_ARB_0298165 successful
TRN_ARB_0298171 successful
TRN_ARB_0298173 successful
TRN_ARB_0298176 successful
TRN_ARB_0298178 successful
T

SYM_ARB_0000970 successful
SYM_ARB_0000977 successful
SYM_ARB_0000991 successful
SYM_ARB_0000999 successful
SYM_ARB_0001000 successful
SYM_ARB_0001003 successful
SYM_ARB_0001006 successful
SYM_ARB_0001009 successful
SYM_ARB_0001013 successful
SYM_ARB_0001025 successful
SYM_ARB_0001043 successful
SYM_ARB_0001049 successful
SYM_ARB_0001061 successful
SYM_ARB_0001081 successful
SYM_ARB_0001146 successful
SYM_ARB_0001151 successful
SYM_ARB_0001156 successful
SYM_ARB_0001160 successful
SYM_ARB_0001163 successful
SYM_ARB_0001173 successful
SYM_ARB_0001187 successful
SYM_ARB_0001192 successful
SYM_ARB_0001193 successful
SYM_ARB_0001195 successful
SYM_ARB_0001196 successful
SYM_ARB_0001198 successful
SYM_ARB_0001204 successful
SYM_ARB_0001205 successful
SYM_ARB_0001225 successful
SYM_ARB_0001226 successful
SYM_ARB_0001239 successful
SYM_ARB_0001246 successful
SYM_ARB_0001257 successful
SYM_ARB_0001272 successful
SYM_ARB_0001273 successful
SYM_ARB_0001280 successful
SYM_ARB_0001286 successful
S

YM7_ARB_0471620 successful
YM7_ARB_0471660 successful
YM7_ARB_0471665 successful
YM7_ARB_0471684 successful
YM7_ARB_0471708 successful
YM7_ARB_0471713 successful
YM7_ARB_0471715 successful
YM7_ARB_0471720 successful
YM7_ARB_0471743 successful
YM7_ARB_0471784 successful
YM7_ARB_0471788 successful
YM7_ARB_0471804 successful
YM7_ARB_0471817 successful
YM7_ARB_0471827 successful
YM7_ARB_0471856 successful
YM7_ARB_0471863 successful
YM7_ARB_0471867 successful
YM7_ARB_0471890 successful
YM7_ARB_0471924 successful
YM7_ARB_0471927 successful
YM7_ARB_0471928 successful
YM7_ARB_0471929 successful
YM7_ARB_0471932 successful
YM7_ARB_0471937 successful
YM7_ARB_0472027 successful
YM7_ARB_0472044 successful
YM7_ARB_0472084 successful
YM7_ARB_0472098 successful
YM7_ARB_0472121 successful
YM7_ARB_0472155 successful
YM7_ARB_0472158 successful
YM7_ARB_0472196 successful
YM7_ARB_0472197 successful
YM7_ARB_0472203 successful
YM7_ARB_0472212 successful
YM7_ARB_0472213 successful
YM7_ARB_0472318 successful
Y

SYM_ARB_0078425 successful
SYM_ARB_0078452 successful
SYM_ARB_0078467 successful
SYM_ARB_0078496 successful
SYM_ARB_0078521 successful
SYM_ARB_0078522 successful
SYM_ARB_0078529 successful
SYM_ARB_0078538 successful
SYM_ARB_0078541 successful
SYM_ARB_0078542 successful
SYM_ARB_0078567 successful
SYM_ARB_0078578 successful
SYM_ARB_0078590 successful
SYM_ARB_0078604 successful
SYM_ARB_0078606 successful
SYM_ARB_0078611 successful
SYM_ARB_0078663 successful
SYM_ARB_0078665 successful
SYM_ARB_0078666 successful
SYM_ARB_0078672 successful
SYM_ARB_0078673 successful
SYM_ARB_0078683 successful
SYM_ARB_0078693 successful
SYM_ARB_0078694 successful
SYM_ARB_0078695 successful
SYM_ARB_0078697 successful
SYM_ARB_0078699 successful
SYM_ARB_0078706 successful
SYM_ARB_0078707 successful
SYM_ARB_0078713 successful
SYM_ARB_0078732 successful
SYM_ARB_0078742 successful
SYM_ARB_0078758 successful
SYM_ARB_0078767 successful
SYM_ARB_0078791 successful
SYM_ARB_0078800 successful
SYM_ARB_0078812 successful
S

TRN_ARB_0282959 successful
TRN_ARB_0282961 successful
TRN_ARB_0282968 successful
TRN_ARB_0282972 successful
TRN_ARB_0282973 successful
TRN_ARB_0282974 successful
TRN_ARB_0282979 successful
TRN_ARB_0282983 successful
TRN_ARB_0282992 successful
TRN_ARB_0282994 successful
TRN_ARB_0283007 successful
TRN_ARB_0283011 successful
TRN_ARB_0283028 successful
TRN_ARB_0283029 successful
TRN_ARB_0283032 successful
TRN_ARB_0283036 successful
TRN_ARB_0283074 successful
TRN_ARB_0283126 successful
TRN_ARB_0283130 successful
TRN_ARB_0283131 successful
TRN_ARB_0283132 successful
TRN_ARB_0283133 successful
TRN_ARB_0283136 successful
TRN_ARB_0283145 successful
TRN_ARB_0283171 successful
MSY_ARB_0092879 successful
MSY_ARB_0092885 successful
MSY_ARB_0092886 successful
MSY_ARB_0092891 successful
MSY_ARB_0092893 successful
MSY_ARB_0092894 successful
MSY_ARB_0092909 successful
MSY_ARB_0092920 successful
MSY_ARB_0092922 successful
MSY_ARB_0092926 successful
MSY_ARB_0092927 successful
MSY_ARB_0092954 successful
M

...

QBS_ARB_0236339 successful
QBS_ARB_0236344 successful
QBS_ARB_0236350 successful
QBS_ARB_0236386 successful
QBS_ARB_0236396 successful
QBS_ARB_0236403 successful
QBS_ARB_0236416 successful
QBS_ARB_0236434 successful
QBS_ARB_0236476 successful
QBS_ARB_0236481 successful
QBS_ARB_0236546 successful
QBS_ARB_0236557 successful
QBS_ARB_0236567 successful
QBS_ARB_0236568 successful
QBS_ARB_0236569 successful
QBS_ARB_0236575 successful
QBS_ARB_0236608 successful
QBS_ARB_0236609 successful
QBS_ARB_0236611 successful
QBS_ARB_0236619 successful
QBS_ARB_0236623 successful
QBS_ARB_0236629 successful
QBS_ARB_0236631 successful
QBS_ARB_0236632 successful
QBS_ARB_0236633 successful
QBS_ARB_0236634 successful
QBS_ARB_0236650 successful
QBS_ARB_0236654 successful
QBS_ARB_0236667 successful
QBS_ARB_0236695 successful
QBS_ARB_0236700 successful
QBS_ARB_0236706 successful
QBS_ARB_0236708 successful
QBS_ARB_0236709 successful
QBS_ARB_0236711 successful
QBS_ARB_0236719 successful
QBS_ARB_0236736 successful
Q

SBN_ARB_0058836 successful
SBN_ARB_0058846 successful
SBN_ARB_0058848 successful
SBN_ARB_0058918 successful
SBN_ARB_0058968 successful
SBN_ARB_0059047 successful
SBN_ARB_0059051 successful
SBN_ARB_0059057 successful
SBN_ARB_0059067 successful
SBN_ARB_0059074 successful
SBN_ARB_0059076 successful
SBN_ARB_0059112 successful
SBN_ARB_0059140 successful
SBN_ARB_0059160 successful
SBN_ARB_0059162 successful
SBN_ARB_0059167 successful
SBN_ARB_0059183 successful
SBN_ARB_0059215 successful
SBN_ARB_0059285 successful
SBN_ARB_0059299 successful
SBN_ARB_0059306 successful
SBN_ARB_0059313 successful
SBN_ARB_0059326 successful
SBN_ARB_0059327 successful
SBN_ARB_0059329 successful
SBN_ARB_0059330 successful
SBN_ARB_0059334 successful
SBN_ARB_0059371 successful
SBN_ARB_0059388 successful
SBN_ARB_0059421 successful
SBN_ARB_0059434 successful
SBN_ARB_0059462 successful
SBN_ARB_0059465 successful
SBN_ARB_0059477 successful
SBN_ARB_0059480 successful
SBN_ARB_0059482 successful
SBN_ARB_0059487 successful
S

MSY_ARB_0044533 successful
MSY_ARB_0044543 successful
MSY_ARB_0044557 successful
MSY_ARB_0044568 successful
MSY_ARB_0044572 successful
MSY_ARB_0044573 successful
MSY_ARB_0044584 successful
MSY_ARB_0044601 successful
MSY_ARB_0044606 successful
MSY_ARB_0044608 successful
MSY_ARB_0044610 successful
MSY_ARB_0044637 successful
MSY_ARB_0044643 successful
MSY_ARB_0044651 successful
MSY_ARB_0044653 successful
MSY_ARB_0044654 successful
MSY_ARB_0044676 successful
MSY_ARB_0044678 successful
MSY_ARB_0044679 successful
MSY_ARB_0044681 successful
MSY_ARB_0044692 successful
MSY_ARB_0044699 successful
MSY_ARB_0044704 successful
MSY_ARB_0044710 successful
MSY_ARB_0044728 successful
MSY_ARB_0044729 successful
MSY_ARB_0044733 successful
MSY_ARB_0044734 successful
MSY_ARB_0044749 successful
MSY_ARB_0044751 successful
MSY_ARB_0044787 successful
MSY_ARB_0044793 successful
MSY_ARB_0044809 successful
MSY_ARB_0044811 successful
MSY_ARB_0044818 successful
MSY_ARB_0044823 successful
MSY_ARB_0044827 successful
M

SYM_ARB_0619594 successful
SYM_ARB_0619596 successful
SYM_ARB_0619608 successful
SYM_ARB_0619625 successful
SYM_ARB_0619659 successful
SYM_ARB_0619703 successful
SYM_ARB_0619797 successful
SYM_ARB_0619829 successful
SYM_ARB_0619830 successful
SYM_ARB_0619831 successful
SYM_ARB_0619833 successful
SYM_ARB_0619939 successful
SYM_ARB_0619942 successful
SYM_ARB_0619948 successful
SYM_ARB_0619963 successful
SYM_ARB_0620010 successful
SYM_ARB_0620011 successful
SYM_ARB_0620014 successful
SYM_ARB_0620025 successful
SYM_ARB_0620028 successful
SYM_ARB_0620029 successful
SYM_ARB_0620039 successful
SYM_ARB_0620040 successful
SYM_ARB_0620041 successful
SYM_ARB_0620044 successful
SYM_ARB_0620081 successful
SYM_ARB_0620098 successful
SYM_ARB_0620170 successful
SYM_ARB_0620224 successful
SYM_ARB_0620230 successful
SYM_ARB_0620238 successful
SYM_ARB_0620240 successful
SYM_ARB_0620241 successful
SYM_ARB_0620242 successful
SYM_ARB_0620244 successful
SYM_ARB_0620248 successful
SYM_ARB_0620249 successful
S

QBS_ARB_0248001 successful
QBS_ARB_0248004 successful
QBS_ARB_0248023 successful
QBS_ARB_0248024 successful
QBS_ARB_0248026 successful
QBS_ARB_0248028 successful
QBS_ARB_0248030 successful
QBS_ARB_0248045 successful
QBS_ARB_0248066 successful
QBS_ARB_0248067 successful
QBS_ARB_0248068 successful
QBS_ARB_0248088 successful
QBS_ARB_0248094 successful
QBS_ARB_0248192 successful
QBS_ARB_0248231 successful
QBS_ARB_0248275 successful
QBS_ARB_0248292 successful
QBS_ARB_0248293 successful
QBS_ARB_0248301 successful
QBS_ARB_0248302 successful
QBS_ARB_0248321 successful
QBS_ARB_0248322 successful
QBS_ARB_0248326 successful
QBS_ARB_0248345 successful
QBS_ARB_0248349 successful
QBS_ARB_0248355 successful
QBS_ARB_0248356 successful
QBS_ARB_0248357 successful
QBS_ARB_0248364 successful
QBS_ARB_0248365 successful
QBS_ARB_0248367 successful
QBS_ARB_0248368 successful
QBS_ARB_0248369 successful
QBS_ARB_0248370 successful
QBS_ARB_0248387 successful
QBS_ARB_0248455 successful
QBS_ARB_0248476 successful
Q

In [7]:
new.head(5)

,index,Unnamed: 0,id,url,headline,dateline,text,tags,source,text_label,text_score,headline_label,headline_score
0,0,274890,TRN_ARB_0297955,http://tishreen.news.sy/tishreen/public/read/2...,وزير الخارجية الصيني: سنقوم بدور بنّاء وإيجابي...,2013-09-20,\n20/09/2013\nوزير الخارجية الصيني: سنقوم بدور...,['واشنطن'],Techreen,neutral,0.9574713706970215,neutral,0.538953
1,1,274891,TRN_ARB_0297957,http://tishreen.news.sy/tishreen/public/read/2...,قواتنا الباسلة تضيّق الخناق على إرهابيي «القاع...,2013-09-20,\n20/09/2013\nضبط مستودع أسلحة للإرهابيين في د...,['القاعدة'],Techreen,False,False,positive,0.899801
2,2,274892,TRN_ARB_0297958,http://tishreen.news.sy/tishreen/public/read/2...,وزير الخارجية الإيراني: انضمام سورية لمعاهدة ح...,2013-09-20,\n20/09/2013\nوزير الخارجية الإيراني: انضمام س...,['واشنطن'],Techreen,neutral,0.5859406590461731,neutral,0.774108
3,3,274893,TRN_ARB_0297960,http://tishreen.news.sy/tishreen/public/read/2...,استمرار أمريكا وحلفائها بدعم وتسليح الإرهابيين...,2013-09-20,\n20/09/2013\n«الخارجية» في رسالتين إلى رئيس م...,"['أمريكا', 'القاعدة']",Techreen,negative,0.9461716612180074,negative,0.957832
4,4,274894,TRN_ARB_0297984,http://tishreen.news.sy/tishreen/public/read/2...,المفاوض الفلسطيني.. والسباحة من جديد في بحر ال...,2013-09-22,\n22/09/2013\nالمفاوض الفلسطيني.. والسباحة من ...,['واشنطن'],Techreen,negative,0.9543005029360453,negative,0.954988


In [8]:
new.columns

Index(['index', 'Unnamed: 0', 'id', 'url', 'headline', 'dateline', 'text',
       'tags', 'source', 'text_label', 'text_score', 'headline_label',
       'headline_score'],
      dtype='object')

In [6]:
new.shape

(23524, 13)

In [9]:
new.info

<bound method DataFrame.info of        index  Unnamed: 0               id  \
0          0      274890  TRN_ARB_0297955   
1          1      274891  TRN_ARB_0297957   
2          2      274892  TRN_ARB_0297958   
3          3      274893  TRN_ARB_0297960   
4          4      274894  TRN_ARB_0297984   
...      ...         ...              ...   
23557    352      121732  QBS_ARB_0250040   
23558    353      121733  QBS_ARB_0250057   
23559    354      121734  QBS_ARB_0250059   
23560    355      121735  QBS_ARB_0250084   
23561    356      121736  QBS_ARB_0250090   

                                                     url  \
0      http://tishreen.news.sy/tishreen/public/read/2...   
1      http://tishreen.news.sy/tishreen/public/read/2...   
2      http://tishreen.news.sy/tishreen/public/read/2...   
3      http://tishreen.news.sy/tishreen/public/read/2...   
4      http://tishreen.news.sy/tishreen/public/read/2...   
...                                                  ...   
23557  

In [11]:
df.headline_label.value_counts()

neutral     16434
negative     5370
positive     1758
Name: headline_label, dtype: int64

In [10]:
df.text_label.value_counts()

neutral     12310
False        6561
negative     3991
positive      700
Name: text_label, dtype: int64

### 1. When to split?? 
### 2. Weird columns
### 3. How to translate?